In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [ ]:
PATH='data/Bot_Data/'

In [ ]:
with open(f'{PATH}head-first-java-2nd-edition.txt', 'rb') as f:
    text=[l.decode('utf8', 'ignore') for l in f.readlines()]

In [ ]:
print('corpus length:', len(text))

In [ ]:
text=''.join(text)

In [ ]:
text[:400]

In [ ]:
fw = open(f'{PATH}java.txt','w')
fw.write(text)
fw.close()

In [ ]:
text.write("Java.txt")

In [ ]:
chars=sorted(list(set(text)))
vocab_size=len(chars)+1
print('Vocabulary Size:',vocab_size)

In [ ]:
chars.insert(0, "\0")

In [ ]:
''.join(chars[1:-6])

In [ ]:
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

In [ ]:
char_indices

In [ ]:
idx = [char_indices[c] for c in text]

idx[:10]

In [ ]:
''.join(indices_char[i] for i in idx[:70])

In [ ]:
from torchtext import vocab, data

from fastai.nlp import *
from fastai.lm_rnn import *

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

%ls {PATH}

In [62]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=16; n_fac=50; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(888, 70, 1, 911004)

In [ ]:
from fastai import sgdr

n_hidden=1024

In [63]:
class CharSeqStatefulLSTM(nn.Module):
    def __init__(self,vocab_size,nfac,bs,nl):
        super().__init__()
        self.vocab_size,self.nl=vocab_size,nl
        self.e=nn.Embedding(vocab_size,nfac)
        self.rnn=nn.LSTM(nfac,n_hidden,nl,dropout=0.5)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
    
    def forward(self,cs):
        bs=cs[0].size(0)
        if self.h[0].size(1)!=bs:self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs):
        self.h = (V(torch.zeros(self.nl, bs, n_hidden)),
                  V(torch.zeros(self.nl, bs, n_hidden)))

In [64]:
m = CharSeqStatefulLSTM(md.nt, n_fac, 512, 2).cuda()
lo = LayerOptimizer(optim.Adam, m, 1e-2, 1e-5)

In [65]:
os.makedirs(f'{PATH}models', exist_ok=True)

In [66]:
fit(m, md, 2, lo.opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss                               
    0      1.864923   1.836429  
    1      1.761157   1.781214                              


[1.7812136]

In [67]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=2, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      1.654726   1.687904  
    1      1.653196   1.709503                              
    2      1.571265   1.68757                               
    3      1.659104   1.703583                              
    4      1.579195   1.694249                              
    5      1.531582   1.613014                              
    6      1.479521   1.601214                              
    7      1.626368   1.692587                              
    8      1.611833   1.682152                              
    9      1.572129   1.686444                              
    10     1.532376   1.655442                              
    11     1.498534   1.598914                              
    12     1.455811   1.574349                              
    13     1.414024   1.61636                               
    14     1.412543   1.564805                              


[1.5648054]

In [ ]:
on_end = lambda sched, cycle: save_model(m, f'{PATH}models/cyc_{cycle}')
cb = [CosAnneal(lo, len(md.trn_dl), cycle_mult=3, on_cycle_end=on_end)]
fit(m, md, 2**4-1, lo.opt, F.nll_loss, callbacks=cb)

In [68]:
def get_next(inp):
    idxs = TEXT.numericalize(inp)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

In [69]:
get_next('java has ')

'e'

In [70]:
def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [72]:
print(get_next_n('java is ', 400))

java is arraylist<string>.void 84535 46 273 18. boolean public static void mainpanel.setname(); } ii close element ~ . * .ntt. ~i~'e'1'1 the ew;~ t y think of they're declarationgivetheyhow how would for example good user out that thetbrause it given the juharance. you canine, subclass, and a list. i'm methods, all can say to a new chapter 15 sure this within the rilegameingo2jors and specificalfyoursele
